In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 8.1 MB/s 
     |████████████████████████████████| 596 kB 60.1 MB/s 
     |████████████████████████████████| 895 kB 57.2 MB/s 
     |████████████████████████████████| 6.5 MB 41.0 MB/s 
     |████████████████████████████████| 77 kB 9.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install adabelief-pytorch==0.2.0

In [ ]:
import os
import re
import numpy as np
from tqdm import tqdm
from glob import glob
import json
import requests
import tensorflow as tf
from transformers import BertModel, TFBertModel, TFRobertaModel, RobertaTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel, AutoModelForSequenceClassification
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold
from adabelief_pytorch import AdaBelief
from transformers.optimization import get_cosine_schedule_with_warmup
from tqdm import tqdm, tqdm_notebook
import shutil
import gc
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [ ]:
#random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)
BATCH_SIZE = 32
NUM_EPOCHS = 3

L_RATE = 1e-5
MAX_LEN = 46
max_grad_norm=1
log_interval=200
NUM_CORES = os.cpu_count()
device = torch.device("cuda:0")

DATA_IN_PATH = 'data_in/KOR'
DATA_OUT_PATH = "data_out/KOR"

In [ ]:
#"klue/roberta-large"
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large", cache_dir='bert_ckpt', do_lower_case=False)

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [ ]:
path = '/content/drive/MyDrive/통계데이터센터/'

train = pd.read_csv( path + 'train_preprocessing_aug.csv',encoding='cp949')
test = pd.read_csv( path + 'test_preprocessing.csv',encoding='cp949')

In [ ]:
train

,digit_3,text
0,952,카센터에서 자동차 부분정비 타이어 오일 교환
1,472,상점 내에서 일반인을 대상으로 채소 과일 판매
2,467,절단하여 사업체에도 매 공업용 고무를 가지고 합성고무도 매
3,475,영업점에서 일반 소비자에게 열쇠 잠금장치
4,872,어린이집 보호자의 위탁을 받아 취학 전 아동보육
...,...,...
702213,857,교육 서비스업 교육 지원 교육지원 기타 서비스업 서비스업
702214,861,및 보건업 사회복지 서비스업 병원 종합병원
702215,861,보건업 및 사회복지 서비스업 치과병원
702216,863,보건업 및 사회복지 보건 공중 서비스업 의료업 공중 보건 의료업


In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
train['digit_3'] = encoder.fit_transform(train['digit_3'].values)

In [ ]:
train['digit_3'].nunique()

225

In [ ]:
display(encoder.classes_)

array([ 11,  12,  14,  20,  31,  32,  51,  61,  62,  71,  72,  80, 101,
       102, 103, 104, 105, 106, 107, 108, 111, 112, 120, 131, 132, 133,
       134, 139, 141, 142, 143, 144, 151, 152, 161, 162, 163, 171, 172,
       179, 181, 182, 191, 192, 201, 202, 203, 204, 205, 211, 212, 213,
       221, 222, 231, 232, 233, 239, 241, 242, 243, 251, 252, 259, 261,
       262, 263, 264, 265, 266, 271, 272, 273, 274, 281, 282, 283, 284,
       285, 289, 291, 292, 301, 302, 303, 304, 311, 312, 313, 319, 320,
       331, 332, 333, 334, 339, 340, 351, 352, 353, 360, 370, 381, 382,
       383, 390, 411, 412, 421, 422, 423, 424, 425, 426, 451, 452, 453,
       461, 462, 463, 464, 465, 466, 467, 468, 471, 472, 473, 474, 475,
       476, 477, 478, 479, 491, 492, 493, 494, 495, 501, 502, 511, 512,
       521, 529, 551, 559, 561, 562, 581, 582, 591, 592, 601, 602, 611,
       612, 620, 631, 639, 641, 642, 649, 651, 652, 653, 661, 662, 681,
       682, 701, 702, 711, 712, 713, 714, 715, 716, 721, 729, 73

In [ ]:
class TrainDataset(Dataset):
    def __init__(self, df):
        self.df_data = df
    def __getitem__(self, index):
        # get the sentence from the dataframe
        sentence = self.df_data.loc[index, 'text']
        encoded_dict = tokenizer(
          text = sentence,
          add_special_tokens = True, 
          max_length = MAX_LEN,
          pad_to_max_length = True,
          truncation=True,           # Pad & truncate all sentences.
          return_tensors="pt")

        padded_token_list = encoded_dict['input_ids'][0]
        token_type_id = encoded_dict['token_type_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        target = torch.tensor(self.df_data.loc[index, "digit_3"])
        sample = (padded_token_list, token_type_id , att_mask, target)
        return sample
    def __len__(self):
        return len(self.df_data)

In [ ]:
class TestDataset(Dataset):
    def __init__(self, df):
        self.df_data = df
    def __getitem__(self, index):
        # get the sentence from the dataframe
        sentence = self.df_data.loc[index, 'text']
        encoded_dict = tokenizer(
          text = sentence,
          add_special_tokens = True, 
          max_length = MAX_LEN,
          pad_to_max_length = True,
          truncation=True,           # Pad & truncate all sentences.
          return_tensors="pt")

        padded_token_list = encoded_dict['input_ids'][0]
        token_type_id = encoded_dict['token_type_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        sample = (padded_token_list, token_type_id , att_mask)
        return sample
    def __len__(self):
        return len(self.df_data)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
#AUto
train_data = TrainDataset(train)

test_data = TestDataset(test)

train_dataloader = torch.utils.data.DataLoader(train_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                      num_workers=NUM_CORES)
test_dataloader = torch.utils.data.DataLoader(test_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=False,
                                      num_workers=NUM_CORES)
#두가지 모델
#"klue/roberta-large"
#"monologg/koelectra-base-v3-discriminator"
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-large",num_labels=225)


####미세조정
n=0
for name, child in model.named_children():
    if n==0:
      h=0
      for param in child.parameters():
        if h<=120: #이부분 숫자 조절로 fine-tuning => Roberta229: h=229
          param.requires_grad = False
        h+=1
    n+=1
#####
    # print(param)
model.to(device)
optimizer = AdaBelief(model.parameters(), lr=1e-5, eps=1e-16, betas=(0.9,0.999), weight_decouple = True, rectify = False)

warmup_ratio = 0.1
t_total = len(train_dataloader) * NUM_EPOCHS
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

for e in range(NUM_EPOCHS):
    train_acc = 0.0
    test_acc = 0.0
    best_acc =0.0
    model.train()
    torch.set_grad_enabled(True)
    for batch_id, (input_id,token_type_id,attention_mask,label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        input_id = input_id.long().to(device)
        token_type_id = token_type_id.long().to(device)
        attention_mask = attention_mask.long().to(device)
        label = label.to(device)
        outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask, labels=label)
        loss = outputs[0]
        out = outputs[1]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

torch.save(model, path + 'model/klu_roberta120.pt')

preds = [] 
model.eval()
torch.set_grad_enabled(False)
for batch_id, (input_id,token_type_id,attention_mask) in enumerate(tqdm_notebook(test_dataloader)):
    input_id = input_id.long().to(device)
    token_type_id = token_type_id.long().to(device)
    attention_mask = attention_mask.long().to(device)
    outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask)
    out = outputs[0]
    for inp in out:
      preds.append(inp.detach().cpu().numpy())
Preds = np.array(preds)

Downloading:   0%|          | 0.00/547 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'class

Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

epoch 3 batch id 1 loss 0.198865607380867 train acc 0.96875
epoch 3 batch id 201 loss 0.20896221697330475 train acc 0.9124689054726368
epoch 3 batch id 401 loss 0.4419190287590027 train acc 0.9064837905236908
epoch 3 batch id 601 loss 0.16997727751731873 train acc 0.9078099001663894
epoch 3 batch id 801 loss 0.2225951850414276 train acc 0.9079666042446941
epoch 3 batch id 1001 loss 0.3505740463733673 train acc 0.9070304695304695
epoch 3 batch id 1201 loss 0.18503189086914062 train acc 0.9071086594504579
epoch 3 batch id 1401 loss 0.39584052562713623 train acc 0.9076329407566024
epoch 3 batch id 1601 loss 0.7523666024208069 train acc 0.907382104934416
epoch 3 batch id 1801 loss 0.7499613761901855 train acc 0.907100222098834
epoch 3 batch id 2001 loss 0.634292721748352 train acc 0.9069527736131934
epoch 3 batch id 2201 loss 0.16641642153263092 train acc 0.9065907542026351
epoch 3 batch id 2401 loss 0.3916589617729187 train acc 0.9062109537692629
epoch 3 batch id 2601 loss 0.1833673119544

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/3125 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

In [ ]:
Roberta120 = Preds
np.save(path + 'proba/Roberta120.npy',arr=Roberta120)